In [ ]:
#create new model (input:onehot-seq;output:peak represent)

In [ ]:
import sys
sys.path.append('/software/path/prefix')

In [ ]:
from ResNeXt_modified import *

In [ ]:
model_new = resnext34()

In [ ]:
model_new

In [ ]:
import h5py, os, argparse, logging, time

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader

In [ ]:
# unpack h5file
h5file = h5py.File('/file/path/prefix/Femalemus_5wCells_17wPeaks_8wnegative.shuffled.noimpute.500bp.20230822.h5', 'r')
X = h5file["pmat"]["X"][:].swapaxes(-1,1).astype(np.float32)
peak_idx = h5file['pmat']['pmat_sc']['i'][:]
cell_idx = h5file['pmat']['pmat_sc']['j'][:]
x = h5file['pmat']['pmat_sc']['x'][:]
dim = h5file['pmat']['pmat_sc']['dim'][:]
y = np.zeros((dim[0], dim[1]), dtype = np.float32)
y[peak_idx, cell_idx] = x
features = h5file["pmat"]["peak"][:]
h5file.close()

In [ ]:
#load para

In [ ]:
load_params = torch.load("./Log/best_model.pth")
model_params = model_new.state_dict()

In [ ]:
model_params.keys()

In [ ]:
# extract the corresponding params
co_params = {k: v for k, v in load_params.items() if k in model_params.keys()}

In [ ]:
co_params.keys()

In [ ]:
model_params.update(co_params)
model_new.load_state_dict(model_params)

In [ ]:
X.shape

In [ ]:
cell_embed=load_params['fc1.dense.weight']

In [ ]:
cell_embed.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_new.eval()

In [ ]:
model_new.to(device)

In [ ]:
w=cell_embed.cpu().numpy()

In [ ]:
import scipy.sparse
from scipy import sparse
import anndata
n_samples = X.shape[0]
from tqdm import tqdm
bs = 100
y_pred = []
for i in tqdm(range(0, n_samples, bs)):#
    X_batch = X[i:i+bs, ...]
    X_batch = torch.from_numpy(X_batch).to(device)
    y_pred_batch = model_new.forward(X_batch).squeeze().cpu().data.numpy()
   # y_pred_batch= np.where(y_pred_batch > t, 1, 0).astype(np.float32)
    y_pred_batch=y_pred_batch.astype(np.float32)
    y_pred_batch=np.dot(y_pred_batch,w.T)
    y_pred_batch=np.divide(1,1+np.exp(-y_pred_batch))
    tmp= np.where(y_pred_batch>0.95, 1, 0).astype(np.float32)
    ad1=anndata.AnnData(tmp)
    ad1.X=sparse.csr_matrix(ad1.X)
    y_pred.append(ad1)

In [ ]:
ad=anndata.concat(y_pred,join='outer')

In [ ]:
ad

In [ ]:
tmp=ad.X.toarray()

In [ ]:
tmp=sparse.csr_matrix(tmp)

In [ ]:
from scipy.io import mmwrite
mmwrite('/file/path/prefix/pmat_pred_imputed_total.mtx',tmp)